In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
base_dir = 'drive/MyDrive/' + <your project dir>

In [ ]:
import torch
print(f"pytorch: {torch.__version__}")
import torch.nn as nn
from torchvision import transforms
from torch.utils.mobile_optimizer import optimize_for_mobile

import matplotlib.pyplot as plt
%matplotlib inline

import pathlib
import random
import os
from PIL import Image

In [ ]:
!git clone https://github.com/ryu38/UGATIT-pytorch-colab.git
!mv UGATIT-pytorch-colab ugatit

from ugatit.models.generator import Generator

In [ ]:
CHANNELS = 3
IMG_SIZE = 256

In [ ]:
model = Generator(n_blocks=8, light=True, simple_output=True).to("cpu")

In [ ]:
load_models_dirname = 'saved_models/' + <pre-trained models dirname>
load_models_filename = <pre-trained models filename>

In [ ]:
!mkdir trained_models
!cp {base_dir}/{load_models_dirname}/{load_models_filename} trained_models/{load_models_filename}
!ls trained_models/

In [ ]:
ckpt = torch.load(os.path.join('trained_models', load_models_filename), map_location=torch.device('cpu'))
model.load_state_dict(ckpt['g_a2b'])

In [ ]:
model = model.eval()

In [ ]:
input_shape = (1, 3, 256, 256)
dummy_input = 2 * torch.rand(input_shape) - 1

In [ ]:
trace = torch.jit.trace(model, dummy_input)

In [ ]:
traced_script_module_optimized = optimize_for_mobile(trace)

In [ ]:
mobile_model_dirname = 'pytorch-mobile'
mobile_model_filename = 'ugatit-mobile.ptl'

!mkdir {mobile_model_dirname}
model_path = f'{mobile_model_dirname}/{mobile_model_filename}'

In [ ]:
traced_script_module_optimized._save_for_lite_interpreter(model_path)

In [ ]:
!cp {model_path} {base_dir}/{model_path}